In [155]:
import pandas as pd

collections = pd.read_csv('collection.tar_1/collection.tsv', sep='\t', nrows=10000, header=None, names=['passage_id', 'passage_text'])
# top1000_dev_df = pd.read_csv('./top1000.dev/top1000.dev', sep='\t', nrows=20000, header=None, names=['passage_id', 'query_id', 'query', 'passage'])


In [157]:
import pandas as pd
chunksize = 5000
chunks = pd.read_csv('./top1000.dev/top1000.dev', sep='\t', header=None, names=['query_id', 'passage_id', 'query', 'passage'], chunksize=chunksize)
top1000_dev_df = pd.DataFrame()

for chunk in chunks:
    filtered_chunk = chunk[chunk['passage_id'] < 10000]
    top1000_dev_df = pd.concat([top1000_dev_df, filtered_chunk])

    # If we have 20000 rows already, stop processing
#     if len(top1000_dev_df) >= 10000:
#         break
# if len(top1000_dev_df) > 10000:
#     top1000_dev_df = top1000_dev_df.iloc[:10000]

In [158]:
query_counts = top1000_dev_df.groupby('query_id')['passage_id'].count().to_dict()

In [159]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.utils import simple_preprocess



stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess(document):
    tokens = simple_preprocess(document)
    # tokens = [ps.stem(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

collections['passage_text_pro'] = [preprocess(document) for document in collections['passage_text']]
# top1000_dev_df['passage_pro'] = [preprocess(document) for document in top1000_dev_df['passage']]
top1000_dev_df['query_pro'] = [preprocess(document) for document in top1000_dev_df['query']]

In [160]:
# Concatenate the 'query_pro' and 'passage_text_pro' columns
# from gensim.models import Word2Vec
# all_documents = pd.concat([top1000_dev_df['query_pro'], collections['passage_text_pro']])
# Tokenize the combined documents
# all_documents = [document.split() for document in all_documents]
# model = Word2Vec(sentences=all_documents, vector_size=1024, window=5, min_count=1, workers=4)
# def get_document_vector(document):
#     tokens = document
#     document_vector = np.mean([model.wv[word] for word in tokens if word in model.wv], axis=0)
#     return document_vector
# passage_documents = [document.split() for document in collections['passage_text_pro']]
# passage_document_vectors = [get_document_vector(document) for document in passage_documents]
# passage_documents = [get_embeddings(query.split()) for query in queries]

import numpy as np
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-300")

def get_embeddings(tokens):
    embeddings = [model[token] for token in tokens if token in model]
    return np.mean(embeddings, axis=0)
passage_document_vectors = [get_embeddings(doc.split()) for doc in collections["passage_text_pro"]]

In [161]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])
if es.indices.exists(index='document_vectors') :
    es.indices.delete(index='document_vectors')

if not es.indices.exists(index="document_vectors"):
    es.indices.create(index='document_vectors')

mapping = {
    "properties": {
        "document_vector": {
            "type": "dense_vector",
            "dims": 300
        }
    }
}
es.indices.put_mapping(body=mapping, index='document_vectors')

C:\Users\HP\AppData\Local\Temp\ipykernel_16236\130317664.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.put_mapping(body=mapping, index='document_vectors')


ObjectApiResponse({'acknowledged': True})

In [162]:
from tqdm import tqdm
from elasticsearch.helpers import parallel_bulk
from collections import deque

actions = []

for i, (document_vector, passage_text) in tqdm(enumerate(zip(passage_document_vectors, collections['passage_text'])), total = len(passage_document_vectors)):
    action = {
        "_op_type": "index",
        "_index": "document_vectors",
        "_id": i,
        "_source": {
            'content': passage_text,
            'document_vector': document_vector
        }
    }
    actions.append(action)

deque(parallel_bulk(es, actions, chunk_size=10000, thread_count=8, queue_size=8), maxlen=0)

100%|██████████| 10000/10000 [00:00<00:00, 547959.87it/s]


deque([])

In [169]:
# from gensim.utils import simple_preprocess 

docs = []
rel = []
setk = []
count = 0
for query, query_id in zip(top1000_dev_df['query_pro'], top1000_dev_df["query_id"]):

  # Tokenize the query
  # embeddings = [model[token] for token in tokens if token in model]
  # query_vector = np.mean(embeddings, axis=0)
  # print([response['hits']['hits']])
  tokens = query.split()
  missing_vec = np.zeros(300)
  embeddings = [model[token] if token in model else missing_vec for token in tokens]
  query_vector = np.mean(embeddings, axis=0)
  script_query = {
    'script_score': {
      'query': {'match_all': {}},
      'script': {
        'source': "cosineSimilarity(params.query_vector, 'document_vector') + 1.0",
        'params': {'query_vector': query_vector.tolist()}  
      }
    }
  }
  response = es.search(index='document_vectors', body={'query': script_query})
  fetched = np.array([int(response['hits']['hits'][i]["_id"]) for i in range(len(response['hits']['hits']))])
  filtered_df = top1000_dev_df[top1000_dev_df['query_pro'] == query]
  docs.append(fetched.tolist())
  rel.append(filtered_df["passage_id"].to_list())
  setk.append(query_counts[query_id])
  if count == 100:
    break
  count += 1

C:\Users\HP\AppData\Local\Temp\ipykernel_16236\1573989050.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index='document_vectors', body={'query': script_query})


In [171]:
def precision_recall_at_k(retrieved, relevant, k):
    retrieved_set = set(retrieved[:k])
    relevant_set = set(relevant)
    retrieved_relevant = retrieved_set.intersection(relevant_set)
    
    precision = len(retrieved_relevant) / len(retrieved_set)
    recall = len(retrieved_relevant) / len(relevant_set)
    
    return precision, recall

pk = [precision_recall_at_k(docs[i], rel[i], len(rel[i])) for i in range(len(docs))]


In [172]:
def average_precision(retrieved, relevant):
    retrieved_set = set(retrieved)
    relevant_set = set(relevant)
    ap = 0.0
    for i in range(len(retrieved)):
        if retrieved[i] in relevant_set:
            precision, _ = precision_recall_at_k(retrieved, relevant, len(relevant))
            ap += precision
    ap /= len(relevant_set)
    return ap

def mean_average_precision(retrieved_list, relevant_list):
    ap_sum = 0.0
    for retrieved, relevant in zip(retrieved_list, relevant_list):
        ap_sum += average_precision(retrieved, relevant)
    map = ap_sum / len(retrieved_list)
    return map


meanAP = mean_average_precision(docs, rel)

In [173]:
from sklearn.metrics import ndcg_score

def compute_ndcg(retrieved, relevant):
    # Transform retrieved and relevant into binary relevance scores
    y_true = [1 if i in relevant else 0 for i in retrieved]
    y_score = [1 if i in relevant else 0 for i in retrieved]
    return ndcg_score([y_true], [y_score])
# compute_ndcg(docs[0], rel[0])
ndgc = [compute_ndcg(docs[i], rel[i]) for i in range(len(docs))]


In [174]:
from tabulate import tabulate


headers = ["Query no", "Precision", "Recall"]
data_with_ndgc = [(i, *data_point) for i, (data_point) in enumerate(pk)]
table = tabulate(data_with_ndgc, headers, tablefmt="grid")

print(table)
print("Mean AP : ", end = '')
print(meanAP)


+------------+-------------+----------+
|   Query no |   Precision |   Recall |
+============+=============+==========+
|          0 |    0.666667 | 0.666667 |
+------------+-------------+----------+
|          1 |    1        | 1        |
+------------+-------------+----------+
|          2 |    0.714286 | 0.714286 |
+------------+-------------+----------+
|          3 |    0.166667 | 0.166667 |
+------------+-------------+----------+
|          4 |    0.166667 | 0.166667 |
+------------+-------------+----------+
|          5 |    0.166667 | 0.166667 |
+------------+-------------+----------+
|          6 |    0.5      | 0.5      |
+------------+-------------+----------+
|          7 |    0.2      | 0.2      |
+------------+-------------+----------+
|          8 |    0.5      | 0.5      |
+------------+-------------+----------+
|          9 |    0        | 0        |
+------------+-------------+----------+
|         10 |    0        | 0        |
+------------+-------------+----------+


In [175]:
from tabulate import tabulate
from termcolor import colored

# Assuming you have your data in the 'headers' and 'data_with_ndgc' variables

# Define colors for header and data rows
header_color = 'yellow'
data_color = 'cyan'

# Colorize the headers
colored_headers = [colored(header, header_color) for header in headers]

# Colorize the data rows
colored_data = [
    [colored(str(cell), data_color) for cell in row]
    for row in data_with_ndgc
]

# Create the colorful table
table = tabulate(colored_data, colored_headers, tablefmt="pretty")

print(table)
print("Mean AP: " + colored(str(meanAP), 'red'))  # You can choose a color for the meanAP as well


+----------+---------------------+---------------------+
| Query no |      Precision      |       Recall        |
+----------+---------------------+---------------------+
|    0     | 0.6666666666666666  | 0.6666666666666666  |
|    1     |         1.0         |         1.0         |
|    2     | 0.7142857142857143  | 0.7142857142857143  |
|    3     | 0.16666666666666666 | 0.16666666666666666 |
|    4     | 0.16666666666666666 | 0.16666666666666666 |
|    5     | 0.16666666666666666 | 0.16666666666666666 |
|    6     |         0.5         |         0.5         |
|    7     |         0.2         |         0.2         |
|    8     |         0.5         |         0.5         |
|    9     |         0.0         |         0.0         |
|    10    |         0.0         |         0.0         |
|    11    |         0.0         |         0.0         |
|    12    | 0.42857142857142855 | 0.42857142857142855 |
|    13    |         0.0         |         0.0         |
|    14    |         0.0       